In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download cdart99/food20dataset

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/cdart99/food20dataset
License(s): CC0-1.0
 99% 432M/435M [00:03<00:00, 147MB/s]
100% 435M/435M [00:03<00:00, 145MB/s]


In [ ]:
! unzip food20dataset.zip

Archive:  food20dataset.zip
  inflating: food20dataset/test_set/biriyani/biriyanitest (1).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (10).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (11).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (12).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (13).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (14).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (15).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (16).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (17).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (18).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (19).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (2).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (20).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (21).jpg  
  inflat

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
        '/content/food20dataset/train_set',
        target_size=(224, 224),
        batch_size=64)
print(train_generator.class_indices)
test_datagen=ImageDataGenerator(rescale=1/255)
test_generator = test_datagen.flow_from_directory(
        '/content/food20dataset/test_set',
        target_size=(224, 224),
        batch_size=64)
print(test_generator.class_indices)
print(test_generator.classes)

Found 1400 images belonging to 20 classes.
{'biriyani': 0, 'bisibelebath': 1, 'butternaan': 2, 'chaat': 3, 'chappati': 4, 'dhokla': 5, 'dosa': 6, 'gulab jamun': 7, 'halwa': 8, 'idly': 9, 'kathi roll': 10, 'meduvadai': 11, 'noodles': 12, 'paniyaram': 13, 'poori': 14, 'samosa': 15, 'tandoori chicken': 16, 'upma': 17, 'vada pav': 18, 'ven pongal': 19}
Found 600 images belonging to 20 classes.
{'biriyani': 0, 'bisibelebath': 1, 'butternaan': 2, 'chaat': 3, 'chappati': 4, 'dhokla': 5, 'dosa': 6, 'gulab jamun': 7, 'halwa': 8, 'idly': 9, 'kathi roll': 10, 'meduvadai': 11, 'noodles': 12, 'paniyaram': 13, 'poori': 14, 'samosa': 15, 'tandoori chicken': 16, 'upma': 17, 'vada pav': 18, 'ven pongal': 19}
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3
  3  3 

In [ ]:
from tensorflow.keras.applications import ResNet50
base_model = ResNet50(
    include_top=False,
    weights = 'imagenet',
    input_shape = (224,224,3)
)
from keras.models import Model,Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPool2D,Flatten
another_model = Sequential()
another_model.add(Flatten())
another_model.add(Dense(1024,activation='relu'))
another_model.add(Dense(512,activation='relu'))
another_model.add(Dense(20,activation='softmax'))
model = Sequential([base_model, another_model])
model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential (Sequential)              │ (None, 20)                  │     103,296,532 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 126,884,244 (484.02 MB)

 Trainable params: 126,831,124 (483.82 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_generator,validation_data = test_generator,epochs=50)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


22/22 ━━━━━━━━━━━━━━━━━━━━ 144s 3s/step - accuracy: 0.0614 - loss: 20.7013 - val_accuracy: 0.0500 - val_loss: 2334.7590
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.1021 - loss: 2.9746 - val_accuracy: 0.0500 - val_loss: 46.2844
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.2113 - loss: 2.6598 - val_accuracy: 0.0483 - val_loss: 3.0139
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.3439 - loss: 2.1657 - val_accuracy: 0.0500 - val_loss: 3.5371
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.5211 - loss: 1.5942 - val_accuracy: 0.0333 - val_loss: 22.0329
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.6545 - loss: 1.1630 - val_accuracy: 0.0500 - val_loss: 5.1946
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.7847 - loss: 0.7028 - val_accuracy: 0.0500 - val_loss: 5.0694
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.9120 - loss: 0.3172 - val_accuracy: 0.0500 - val_loss: 4.5

In [ ]:
model.evaluate(test_generator)

10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 902ms/step - accuracy: 0.6106 - loss: 2.7520


[2.7241029739379883, 0.6133333444595337]

In [ ]:
model.save('food_model.h5')